In [1]:
def log_loss(p, stim, resp, order):
    [Nsamples,Ndim] = np.shape(stim)
    ptemp = p[1:Ndim+1]

    if order>1:
        J = np.transpose(p[Ndim+1:].reshape([Ndim,Ndim]))
    if order==1:
        f1 = 1 + np.exp(p[0] + np.dot(stim,np.transpose(ptemp)))
        f0 = 1 + np.exp(-p[0] - np.dot(stim,np.transpose(ptemp)))
    else:
        f1 = 1 + np.exp(p[0] + np.dot(stim,np.transpose(ptemp)) + np.sum(np.multiply(stim,np.dot(stim,J)), axis=1))
        f0 = 1 + np.exp(-p[0] - np.dot(stim,np.transpose(ptemp)) - np.sum(np.multiply(stim,np.dot(stim,J)), axis=1))

    Nspikeperstim = resp    # Nsamples x 1
    F1 = np.multiply(Nspikeperstim, np.log10(f1))
    F0 = np.multiply((1 - Nspikeperstim), np.log10(f0))
    F1[np.isnan(F1)] = 0
    F0[np.isnan(F0)] = 0
    f = np.mean(F0 + F1) 
    return f

In [2]:
def d_log_loss(p,stim,avgs,order):
    [Nsamples,Ndim] = np.shape(stim)
    ptemp = p[1:Ndim+1]
    averages = np.empty(np.shape(pstart))

    if order>1:
        J = p[Ndim+1:].reshape([Ndim,Ndim])
    if order==1:
        pSpike = np.divide(1,1 + np.exp(p[0] + np.dot(stim,np.transpose(ptemp))))   #Nsamples x 1
        averages[0] = np.mean(pSpike)
        averages[1:Ndim+1] = np.dot(np.transpose(stim),pSpike)/Nsamples
    else:
        pSpike = np.divide(1,1 + np.exp(p[0] + np.dot(stim,np.transpose(ptemp)) + np.sum(np.multiply(stim,np.dot(stim,J)), axis=1)))
        averages[0] = np.mean(pSpike)
        averages[1:Ndim+1] = np.divide(np.dot(np.transpose(stim),pSpike),Nsamples)
        temp1 = np.transpose(np.matlib.repmat(pSpike,Ndim,1))
        temp = np.divide(np.dot(np.transpose(stim),np.multiply(temp1,stim)),Nsamples)
        temp = temp.flatten()
        averages[Ndim+1:] = temp

    df = np.transpose(avgs - averages)  # 1 x Ndim
    return df

In [ ]:
func = log_Loss
fdunc = d_log_Loss


In [ ]:
def dlinmin():
    TOL = 2.0e-4
    nrfunc = func 
    nrdfun = dfunc 
    pcom   = p 
    xicom  = xi 
    
    ax = 0.0       #Initial guess for brackets
    xx = 0.2       #2*rand();%2.0;
    
    ftemp = @f1dim
    ftemp2 = @df1dim
    
    [ax, xx, bx, fa1, fc1, fb1] = mnbrak(ax, xx, ftemp, stim, resp, order, avgs)
    #[ax,xx,bx]
    #plotalonglinemin(ax,bx,ftemp)
    [fret, xmin] = dbrent(ax,xx,bx,ftemp,ftemp2,TOL, stim, resp, order, avgs)
    #xmin
    #plot([xmin],[fret],'ro','LineWidth',7)
    #drawnow
    #hold off
    xi = xi.*xmin
    p = p + xi
    

In [ ]:
function [p, xi, fret] = dlinmin(p, xi, func, dfunc, stim, resp, order, avgs)
 
TOL    = 2.0e-4;  % Tolerance passed to brent. 

global pcom xicom nrfunc nrdfun; 
nrfunc = func; 
nrdfun = dfunc; 
pcom   = p; 
xicom  = xi; 
 
ax     = 0.0;  % Initial guess for brackets. 
xx     = .2;%2*rand();%2.0; 

ftemp=@f1dim;
ftemp2=@df1dim;

[ax, xx, bx, fa1, fc1, fb1] = mnbrak(ax, xx, ftemp, stim, resp, order, avgs);
%[ax,xx,bx]
%plotalonglinemin(ax,bx,ftemp);
[fret, xmin] = dbrent(ax,xx,bx,ftemp,ftemp2,TOL, stim, resp, order, avgs); 
%xmin
%plot([xmin],[fret],'ro','LineWidth',7); 
%drawnow;
%hold off;
xi     = xi.*xmin;
p      = p + xi;

In [3]:
GOLD=1.618034
GLIMIT=100.0
TINY=1.0e-20

fa = f(ax, stim, resp, order)


In [3]:
def mnbrak():
    #brackets a minimum of f
    #must be supplied with f=@x
    
    #Constants
    GOLD=1.618034
    GLIMIT=100.0
    TINY=1.0e-20
    
    fa = f(ax, stim, resp, order)
    fb = f(bx, stim, resp, order)
    
    if fb>fa:
        dum=ax
        ax=bx
        bx=dum
        dum=fb
        fb=fa
        fa=dum
        
    cx = bx + GOLD*(bx-ax)
    fc = f(cx, stim, resp, order)
    
    while fb>fc:
        r = (bx-ax)*(fb-fc)
        q = (bx-cx)*(fb-fa)
        u = bx - ((bx-cx))*q - (bx-ax)*r)/(2.0*max([abs(q-r),TINY])*sign(q-r))
        ulim = (bx) + GLIMIT*(cx-bx)
        if (bx-u)*(u-cx)>0.0:
            fu = f(u, stim, resp, order)
            if fu < fc:
                ax=bx
                bx=u
                fa=fb
                fb=fu
            else if fu > fb:
                cx = u
                fc = fu
            u = cx + GOLD*(cx-bx)
            fu = f(u, stim, resp, order)
        else if (cx-u)*(u-ulim) > 0.0:
            fu = f(u, stim, resp, order)
            
        
        
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-3-fbb8189bd54e>, line 1)

In [ ]:
function [ax, bx, cx, fa, fb, fc]=mnbrak(ax,bx,f, stim, resp, order, avgs)
%brackets a minimum of f
%must be supplied with f=@(x) 

% Constants
GOLD=1.618034;
GLIMIT=100.0;
TINY=1.0e-20;

fa=f(ax, stim, resp, order);
fb=f(bx, stim, resp, order);

if fb>fa,
    dum=ax ; ax=bx ; bx=dum ;
    dum=fb ; fb=fa ; fa=dum ;
end;

cx=bx+GOLD*(bx-ax);
fc=f(cx, stim, resp, order);

while fb>fc
    r=(bx-ax)*(fb-fc);
    q=(bx-cx)*(fb-fa);
    u= bx -((bx-cx)*q-(bx-ax)*r)/(2.0*max([abs(q-r),TINY])*sign(q-r));
    ulim=(bx)+GLIMIT*(cx-bx);
    if (bx-u)*(u-cx)>0.0,
        fu=f(u, stim, resp, order) ;
        if (fu < fc)
            ax=bx;
            bx=u;
            fa=fb;
            fb=fu;
            return;
        elseif (fu> fb)
            cx=u;
            fc=fu;
            return;
        end
        u=cx+GOLD*(cx-bx);
        fu=f(u, stim, resp, order);
    elseif (cx-u)*(u-ulim)>0.0,
        fu=f(u, stim, resp, order) ;
        if fu<fc
            bx=cx;
            cx=u;
            u=u+GOLD*(u-cx);
            fb=fc;
            fc=fu;
            fu=f(u, stim, resp, order);
        end
    elseif ((u-ulim)*(ulim-cx))>=0.0
        u=ulim;
        fu=f(u, stim, resp, order);
    else
        u=(cx)+GOLD*(cx-bx);
        fu=f(u, stim, resp, order) ;
    end ;
    
    ax=bx ; bx=cx ; cx=u ;
    fa=fb ; fb=fc ; fc=fu ;
end